# Disparate Impact by Providers' Gender 
## the best model: XGBoost

In [2]:
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import glob
import copy
from collections import Counter
from numpy import where
import statsmodels.api as sm
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
import random
import itertools
from interpret.glassbox import ExplainableBoostingClassifier 
import xgboost as xgb
from interpret.perf import ROC  
from imblearn import over_sampling
from imblearn import under_sampling
from imblearn.pipeline import Pipeline
import os              # for directory and file manipulation
import numpy as np     # for basic array manipulation
import pandas as pd    # for dataframe manipulation
import datetime        # for timestamp

# for model eval
from sklearn.metrics import accuracy_score, f1_score, log_loss, mean_squared_error, roc_auc_score

# global constants 
ROUND = 3    

# set global random seed for better reproducibility
SEED = 1234
seed = 1234
NTHREAD = 4

#import sagemaker, boto3, os

import warnings
warnings.filterwarnings('ignore')


In [45]:
# import the cleaned dataset containing Gender feature

#%cd /Users/alex/Desktop/Master/BA_Practicum_6217_10/Project/dataset
partB = pd.read_csv("partB_new5.csv")

In [46]:
partB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348887 entries, 0 to 348886
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   NPI                 348887 non-null  int64  
 1   Gender              348887 non-null  object 
 2   Type                348887 non-null  object 
 3   Place_Of_Srvc       348887 non-null  object 
 4   Tot_Benes           348887 non-null  int64  
 5   Tot_Srvcs           348887 non-null  float64
 6   Tot_Bene_Day_Srvcs  348887 non-null  int64  
 7   Avg_Sbmtd_Chrg      348887 non-null  float64
 8   Avg_Mdcr_Alowd_Amt  348887 non-null  float64
 9   Avg_Mdcr_Pymt_Amt   348887 non-null  float64
 10  Avg_Mdcr_Stdzd_Amt  348887 non-null  float64
 11  Fraud               348887 non-null  int64  
dtypes: float64(5), int64(4), object(3)
memory usage: 31.9+ MB


In [47]:
# One-Hot Encoding 

# Convert the Fraud variable to object datatype
partB["Fraud"] = partB["Fraud"].astype(object)

# Encoding
encoded_partB = pd.get_dummies(partB, drop_first = True)

# Rename some of the changed variable names
encoded_partB.rename(columns = {"Gender_M":"Gender", "Fraud_1":"Fraud", "Place_Of_Srvc_O":"Place_Of_Srvc"}, inplace = True)

## Data Partitioning

In [48]:
# Assign X and y features

X_var = list(encoded_partB.columns)

for var in ["NPI","Fraud"]:
    X_var.remove(var)

y_var = "Fraud"

In [49]:
# Split the whole dataset into train and test dataset
# Using a stratified random sampling so that the Fraud-class (1) data are evenly split into train & test sets
x_train, x_test, y_train, y_test = train_test_split(encoded_partB[X_var], 
                                                    encoded_partB[y_var], 
                                                    test_size=0.2, 
                                                    stratify=encoded_partB["Fraud"])

# Also concatenate the split x & y dataframes 
tr_df = pd.concat([x_train, y_train], axis = 1)
te_df = pd.concat([x_test, y_test], axis = 1)

## Over-Sampling

In [50]:
# SMOTE the dataset
oversample = over_sampling.SMOTE()
tr_X, tr_y = oversample.fit_resample(tr_df[X_var], tr_df[y_var])

## Modeling

### Data Partitioning (Train & Valid)

In [36]:
trans_tr_df = pd.concat([tr_X, tr_y], axis = 1)

# Split train and validation sets 
np.random.seed(SEED)

ratio = 0.7 # split train & validation sets with 7:3 ratio 

split = np.random.rand(len(trans_tr_df)) < ratio # define indices of 70% corresponding to the training set

train = trans_tr_df[split]
valid = trans_tr_df[~split]

# summarize split
print('Train data rows = %d, columns = %d' % (train.shape[0], train.shape[1]))
print('Validation data rows = %d, columns = %d' % (valid.shape[0], valid.shape[1]))

Train data rows = 355544, columns = 117
Validation data rows = 151926, columns = 117


In [37]:
# reassign X_var
X_var.remove("Gender")

### XGBM

In [38]:
def xgb_grid(dtrain, dvalid, mono_constraints=None, gs_params=None, n_models=None,
             ntree=None, early_stopping_rounds=None, verbose=False, seed=None):
    
    """ Performs a random grid search over n_models and gs_params.

    :param dtrain: Training data in LightSVM format.
    :param dvalid: Validation data in LightSVM format.
    :param mono_constraints: User-supplied monotonicity constraints.
    :param gs_params: Dictionary of lists of potential XGBoost parameters over which to search.
    :param n_models: Number of random models to evaluate.
    :param ntree: Number of trees in XGBoost model.
    :param early_stopping_rounds: XGBoost early stopping rounds.
    :param verbose: Whether to display training iterations, default False.
    :param seed: Random seed for better interpretability.
    :return: Best candidate model from random grid search.

    """

    # cartesian product of gs_params
    keys, values = zip(*gs_params.items())
    experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]

    # preserve exact reproducibility for this function
    np.random.seed(SEED) 
    
    # select randomly from cartesian product space
    selected_experiments = np.random.choice(len(experiments), n_models)

    # set global params for objective,  etc.
    params = {'booster': 'gbtree',
              'eval_metric': 'auc',
              'nthread': NTHREAD,
              'objective': 'binary:logistic',
              'seed': SEED}

    # init grid search loop
    best_candidate = None
    best_score = 0

    # grid search loop
    for i, exp in enumerate(selected_experiments):

        params.update(experiments[exp])  # override global params with current grid run params

        print('Grid search run %d/%d:' % (int(i + 1), int(n_models)))
        print('Training with parameters:', params)

        # train on current params
        watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
        
        if mono_constraints is not None:
            params['monotone_constraints'] = mono_constraints
        
        candidate = xgb.train(params,
                              dtrain,
                              ntree,
                              early_stopping_rounds=early_stopping_rounds,
                              evals=watchlist,
                              verbose_eval=verbose)    

        # determine if current model is better than previous best
        if candidate.best_score > best_score:
            best_candidate = candidate
            best_score = candidate.best_score
            print('Grid search new best score discovered at iteration %d/%d: %.4f.' %
                             (int(i + 1), int(n_models), candidate.best_score))

        print('---------- ----------')
            
    return best_candidate

In [39]:
gs_params = {'colsample_bytree': [0.7],
             'colsample_bylevel': [0.9],
             'eta': [0.5],
             'max_depth': [7], 
             'reg_alpha': [0.005],
             'reg_lambda': [0.005],
             'subsample': [0.9],
             'min_child_weight': [1], 
             'gamma': [0.2]}

# Convert data to SVMLight format
dtrain = xgb.DMatrix(train[X_var], train[y_var])
dvalid = xgb.DMatrix(valid[X_var], valid[y_var])

best_mxgb = xgb_grid(dtrain, dvalid, gs_params=gs_params, n_models=1, ntree=1000, early_stopping_rounds=100, seed=SEED)

Grid search run 1/1:
Training with parameters: {'booster': 'gbtree', 'eval_metric': 'auc', 'nthread': 4, 'objective': 'binary:logistic', 'seed': 1234, 'colsample_bytree': 0.7, 'colsample_bylevel': 0.9, 'eta': 0.5, 'max_depth': 7, 'reg_alpha': 0.005, 'reg_lambda': 0.005, 'subsample': 0.9, 'min_child_weight': 1, 'gamma': 0.2}
Grid search new best score discovered at iteration 1/1: 0.9840.
---------- ----------


### Combine valid set with the best prediction


In [40]:
dtest = xgb.DMatrix(te_df[X_var])
best_mxgb_phat = pd.DataFrame(best_mxgb.predict(dtest, iteration_range=(0, best_mxgb.best_ntree_limit)), columns=['phat'])
best_mxgb_phat = pd.concat([te_df.reset_index(drop=True), best_mxgb_phat], axis=1)
best_mxgb_phat.head()

,Tot_Benes,Tot_Srvcs,Tot_Bene_Day_Srvcs,Avg_Sbmtd_Chrg,Avg_Mdcr_Alowd_Amt,Avg_Mdcr_Pymt_Amt,Avg_Mdcr_Stdzd_Amt,Gender,Type_Advanced Heart Failure and Transplant Cardiology,Type_Allergy/ Immunology,...,Type_Thoracic Surgery,Type_Undefined Physician type,Type_Undersea and Hyperbaric Medicine,Type_Unknown Physician Specialty Code,Type_Unknown Supplier/Provider,Type_Urology,Type_Vascular Surgery,Place_Of_Srvc,Fraud,phat
0,11,12.0,12,20.000000,11.690000,11.690000,16.030000,0,0,0,...,0,0,0,0,0,0,0,1,0,0.269147
1,100,163.0,163,33.000000,11.620184,11.620184,11.620184,0,0,0,...,0,0,0,0,0,0,0,1,0,0.566231
2,159,159.0,159,1665.635220,219.603396,174.243208,176.952201,1,0,0,...,0,0,0,0,0,0,0,0,0,0.001677
3,185,193.0,193,174.147668,50.593990,38.662591,39.153990,1,0,0,...,0,0,0,0,0,0,0,0,0,0.018855
4,111,175.0,175,231.000000,140.234686,105.528400,109.437200,0,0,0,...,0,0,0,0,0,0,0,1,0,0.074380


## Mitigating Discrimination 

### Utility functions 
### Calculate confusion matrices by demographic group

In [41]:
def get_confusion_matrix(frame, y, yhat, by=None, level=None, cutoff=0.2, verbose=True):

    """ Creates confusion matrix from pandas dataframe of y and yhat values, can be sliced 
        by a variable and level.
    
        :param frame: Pandas dataframe of actual (y) and predicted (yhat) values.
        :param y: Name of actual value column.
        :param yhat: Name of predicted value column.
        :param by: By variable to slice frame before creating confusion matrix, default None.
        :param level: Value of by variable to slice frame before creating confusion matrix, default None.
        :param cutoff: Cutoff threshold for confusion matrix, default 0.5. 
        :param verbose: Whether to print confusion matrix titles, default True. 
        :return: Confusion matrix as pandas dataframe. 
        
    """
    
    # determine levels of target (y) variable
    # sort for consistency
    level_list = list(frame[y].unique())
    level_list.sort(reverse=True) 

    # init confusion matrix
    cm_frame = pd.DataFrame(columns=['actual: ' +  str(i) for i in level_list], 
                            index=['predicted: ' + str(i) for i in level_list])
    
    # don't destroy original data
    frame_ = frame.copy(deep=True)
    
    # convert numeric predictions to binary decisions using cutoff
    dname = 'd_' + str(y)
    frame_[dname] = np.where(frame_[yhat] > cutoff , 1, 0)
    
    # slice frame
    if (by is not None) & (level is not None):
        frame_ = frame_[frame[by] == level]
    
    # calculate size of each confusion matrix value
    for i, lev_i in enumerate(level_list):
        for j, lev_j in enumerate(level_list):
            cm_frame.iat[j, i] = frame_[(frame_[y] == lev_i) & (frame_[dname] == lev_j)].shape[0]
            # i, j vs. j, i nasty little bug ... updated 8/30/19
    
    # output results
    if verbose:
        if by is None:
            print('Confusion matrix:')
        else:
            print('Confusion matrix by ' + by + '=' + str(level))
    
    return cm_frame

### Calculate Adverse Impact Ratio (AIR) 

In [42]:
def air(cm_dict, reference_key, protected_key, verbose=True):

    """ Calculates the adverse impact ratio as a quotient between protected and 
        reference group acceptance rates: protected_prop/reference_prop. 
        Optionally prints intermediate values. ASSUMES 0 IS "POSITIVE" OUTCOME!

        :param cm_dict: Dictionary of demographic group confusion matrices. 
        :param reference_key: Name of reference group in cm_dict as a string.
        :param protected_key: Name of protected group in cm_dict as a string.
        :param verbose: Whether to print intermediate acceptance rates, default True. 
        :return: AIR.
        
    """

    eps = 1e-20 # numeric stability and divide by 0 protection
    
    # reference group summary
    reference_accepted = float(cm_dict[reference_key].iat[1,0] + cm_dict[reference_key].iat[1,1]) # predicted 0's
    reference_total = float(cm_dict[reference_key].sum().sum())
    reference_prop = reference_accepted/reference_total
    if verbose:
        print(reference_key.title() + ' proportion accepted: %.3f' % reference_prop)
    
    # protected group summary
    protected_accepted = float(cm_dict[protected_key].iat[1,0] + cm_dict[protected_key].iat[1,1]) # predicted 0's
    protected_total = float(cm_dict[protected_key].sum().sum())
    protected_prop = protected_accepted/protected_total
    if verbose:
        print(protected_key.title() + ' proportion accepted: %.3f' % protected_prop)

    # return adverse impact ratio
    return ((protected_prop + eps)/(reference_prop + eps))

### Select Probability Cutoff by F1-score

In [43]:
def get_max_f1_frame(frame, y, yhat, res=0.01, air_reference=None, air_protected=None): 
    
    """ Utility function for finding max. F1. 
        Coupled to get_confusion_matrix() and air(). 
        Assumes 1 is the marker for class membership.
    
        :param frame: Pandas dataframe of actual (y) and predicted (yhat) values.
        :param y: Known y values.
        :param yhat: Model scores.
        :param res: Resolution over which to search for max. F1, default 0.01.
        :param air_reference: Reference group for AIR calculation, optional.
        :param air_protected: Protected group for AIR calculation, optional.
        :return: Pandas DataFrame of cutoffs to select from.
    
    """
    
    do_air = all(v is not None for v in [air_reference, air_protected])
    
    # init frame to store f1 at different cutoffs
    if do_air:
        columns = ['cut', 'f1', 'acc', 'air']
    else:
        columns = ['cut', 'f1', 'acc']
    f1_frame = pd.DataFrame(columns=['cut', 'f1', 'acc'])
    
    # copy known y and score values into a temporary frame
    temp_df = frame[[y, yhat]].copy(deep=True)
    
    # find f1 at different cutoffs and store in acc_frame
    for cut in np.arange(0, 1 + res, res):
        temp_df['decision'] = np.where(temp_df.iloc[:, 1] > cut, 1, 0)
        f1 = f1_score(temp_df.iloc[:, 0], temp_df['decision'])
        acc = accuracy_score(temp_df.iloc[:, 0], temp_df['decision'])
        row_dict = {'cut': cut, 'f1': f1, 'acc': acc}
        if do_air:
            # conditionally calculate AIR  
            cm_ref = get_confusion_matrix(frame, y, yhat, by=air_reference, level=1, cutoff=cut, verbose=False)
            cm_pro = get_confusion_matrix(frame, y, yhat, by=air_protected, level=1, cutoff=cut, verbose=False)
            air_ = air({air_reference: cm_ref, air_protected: cm_pro}, air_reference, air_protected, verbose=False)
            row_dict['air'] = air_
            
        f1_frame = f1_frame.append(row_dict, ignore_index=True)
            
    del temp_df
        
    return f1_frame

### Find optimal cutoff based on F1


In [44]:
f1_frame = get_max_f1_frame(best_mxgb_phat, y_var, 'phat')

print(f1_frame)
print()

max_f1 = f1_frame['f1'].max()
best_cut = f1_frame.loc[int(f1_frame['f1'].idxmax()), 'cut'] #idxmax() returns the index of the maximum value
acc = f1_frame.loc[int(f1_frame['f1'].idxmax()), 'acc']

print('Best XGB F1: %.4f achieved at cutoff: %.2f with accuracy: %.4f.' % (max_f1, best_cut, acc))

      cut        f1       acc
0    0.00  0.166656  0.090903
1    0.01  0.283816  0.587520
2    0.02  0.313475  0.661283
3    0.03  0.332788  0.702041
4    0.04  0.346562  0.728869
..    ...       ...       ...
96   0.96  0.113098  0.912566
97   0.97  0.101147  0.912379
98   0.98  0.084402  0.912007
99   0.99  0.059897  0.911376
100  1.00  0.000000  0.909097

[101 rows x 3 columns]

Best XGB F1: 0.4062 achieved at cutoff: 0.21 with accuracy: 0.8587.


### Specify Interesting Demographic Groups

In [51]:
best_mxgb_phat_copy = best_mxgb_phat.copy()
best_mxgb_phat_copy.rename(columns = {"Gender":"male"}, inplace = True)
best_mxgb_phat_copy["female"] = np.where(best_mxgb_phat_copy["male"] == 0, 1,0)

### Confusion Matrix by Groups

In [52]:
demographic_group_names = ['male', 'female']
cm_dict = {}

for name in demographic_group_names:
    cm_dict[name] = get_confusion_matrix(best_mxgb_phat_copy, y_var, 'phat', by=name, level=1, cutoff=best_cut)
    print(cm_dict[name])
    print()

Confusion matrix by male=1
             actual: 1 actual: 0
predicted: 1      3185      5133
predicted: 0      2726     39566

Confusion matrix by female=1
             actual: 1 actual: 0
predicted: 1       187      1755
predicted: 0       245     16981



### Find AIR for Female people
* protect accepted: female providers
* reference accepted: male providers

In [53]:
print('Adverse impact ratio(AIR) for Females vs. Males: %.3f' % air(cm_dict, 'male', 'female'))

Male proportion accepted: 0.836
Female proportion accepted: 0.899
Adverse impact ratio(AIR) for Females vs. Males: 1.075


* Threshold: AIR >= 0.8 